In [1]:
import os
import glob
import pandas as pd
def get_data(dn):
    data = {
        "news":[],
        "ans":[]
    }
    
    dirs = glob.glob(os.path.join(dn, "*"))
    for d in dirs:
        pat = os.path.join(d, "*.[tT][xX][tT]")
        fs = glob.glob(pat)
        for fp in fs:
            with open(fp, "r", encoding="utf-8") as f:
                news = f.read()
            ans = os.path.split(d)[-1]
            data["news"].append(news)
            data["ans"].append(ans)
    df = pd.DataFrame(data, columns={"news", "ans"})
    return df

In [2]:
train_df = get_data("chinese_news_trans")
test_df = get_data("chinese_news_test")
test_df

,news,ans
0,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
1,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
2,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
3,三趟火車停開 乘客可全額退票 \n瀏覽次數：1180 \n 昨日，來自鐵路部門的消息說...,交通
4,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
...,...,...
96,最優秀選手無緣亞運會健美賽\n \n \n 健美在亞洲運動會上是“新生兒”——韓國釜...,體育
97,\n各國記者眼中的羽毛球世錦賽\n \n\n-----------------------...,體育
98,\n友好運動會第五天 東道主選手大顯神威\n2001年09月03日02:51:40 新華社 ...,體育
99,不靠技術比運氣\n\n 第二屆奧運會在巴黎舉行，同時這裡也正在舉行國際博覽會，東道主把一些...,體育


In [3]:
# 準備答案的對照

#unique value_counts
kind = train_df["ans"].unique()
trans = {k:i for i, k in enumerate(kind)}
reverse_trans = {k:v for v,k in trans.items()}
reverse_trans

{0: '交通',
 1: '政治',
 2: '教育',
 3: '環境',
 4: '經濟',
 5: '藝術',
 6: '計算機',
 7: '軍事',
 8: '醫藥',
 9: '體育'}

In [4]:
# 準備答案
y_train = train_df["ans"].replace(trans)
y_test = test_df["ans"].replace(trans)
y_test

0      0
1      0
2      0
3      0
4      0
      ..
96     9
97     9
98     9
99     9
100    9
Name: ans, Length: 101, dtype: int64

In [6]:
# 下載大字典
import os
import jieba
from urllib.request import urlretrieve

if not os.path.exists("dict.txt.big"):
    print("[1st time] 下載大辭典")
    url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
    urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")


In [7]:

def newscut(s):
    s = " ".join(jieba.cut(s))
    s = s.replace("\r", "").replace("\n", "")
    return s
train = train_df["news"].apply(newscut)
test = test_df["news"].apply(newscut)
test

Building prefix dict from C:\Users\user\PycharmProjects\MLDemo1\dict.txt.big ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.u1d26fb170b1408873e0250faa96804c8.cache
Loading model cost 1.162 seconds.
Prefix dict has been built successfully.


0      日月 光華   --   Traffic _ Info 精華區 文章 閱讀  -------...
1      日月 光華   --   Traffic _ Info 精華區 文章 閱讀  -------...
2      日月 光華   --   Traffic _ Info 精華區 文章 閱讀  -------...
3      三趟 火車 停開   乘客 可 全額 退票    瀏覽 次數 ： 1180         ...
4      日月 光華   --   Traffic _ Info 精華區 文章 閱讀  -------...
                             ...                        
96     最 優秀 選手 無緣 亞運會 健美 賽     　            健美 在 亞洲 運...
97        各國 記者 眼中 的 羽毛球 世錦賽      -------------------...
98      友好 運動會 第五天   東道主 選手 大顯神威  2001 年 09 月 03 日 02...
99     不靠 技術 比 運氣   　 　 第二屆 奧運會 在 巴黎 舉行 ， 同時 這裡 也 正在 ...
100    帆   板   運   動   簡   介 （ 二 ）   　 　 我國 在 79 年 由 ...
Name: news, Length: 101, dtype: object

In [8]:
# 計算次數
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train = vec.fit_transform(train)
x_test = vec.transform(test)

In [9]:
x_test

<101x76399 sparse matrix of type '<class 'numpy.int64'>'
	with 27152 stored elements in Compressed Sparse Row format>

In [10]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [11]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test)
accuracy_score(pre, y_test)

1.0

In [12]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
ori = [k for k in trans.keys()]
ori
c = ["{}(預測)".format(s) for s in ori]
i = ["{}(正確)".format(s) for s in ori]
pd.DataFrame(mat,
             columns=c,
             index=i)

,交通(預測),政治(預測),教育(預測),環境(預測),經濟(預測),藝術(預測),計算機(預測),軍事(預測),醫藥(預測),體育(預測)
交通(正確),10,0,0,0,0,0,0,0,0,0
政治(正確),0,11,0,0,0,0,0,0,0,0
教育(正確),0,0,10,0,0,0,0,0,0,0
環境(正確),0,0,0,10,0,0,0,0,0,0
經濟(正確),0,0,0,0,10,0,0,0,0,0
藝術(正確),0,0,0,0,0,10,0,0,0,0
計算機(正確),0,0,0,0,0,0,10,0,0,0
軍事(正確),0,0,0,0,0,0,0,10,0,0
醫藥(正確),0,0,0,0,0,0,0,0,10,0
體育(正確),0,0,0,0,0,0,0,0,0,10


In [25]:
s = input("請輸入新聞:")
test = vec.transform([newscut(s)])
pre = clf.predict(test)[0]
ans = reverse_trans[pre]
print("新聞的種類:", ans)

proba = clf.predict_proba(test)[0]

proba = list(enumerate(proba))
proba

for i, prob in sorted(proba, key=lambda x:x[1]):
    print(reverse_trans[i], ":", round(prob*100, 2), "%")

請輸入新聞:新冠肺炎疫情衝擊，美國總統川普說自己有絕對權力決定何時撤銷禁足令。對此，紐約州長古莫質疑，「絕對權力」的說法違背憲法與民主觀念，強調美國不需要國王，他不會在危害全州人民健康的情況下重啟經濟。 (特約記者戚海倫報導)     紐約州長古莫說，他無意槓上川普，也願意與川普合作，「如果他想打架，我不會隨之起舞，因為我們眼前有艱鉅的挑戰、需要攜手面對。」     紐約州是美國這次疫情的重災區，全美有超過2萬7000人確診死亡，而紐約州就佔了1萬8000人。紐約州長古莫今天表示，根據住院患者總數和重症患者數據減少看來，他們認為疫情已經達到高峰。     對抗疫情和重啟經濟的時機考驗著美國。古莫昨天和美東其他六州的州長宣布，研議區域經濟重啟方案，避免操之過急導致疫情再起，美西包括加州在內有3個州也宣布，將協調重啟經濟，這一共10個州的州長有9人是民主黨籍。     不過總統川普宣稱，聯邦政府有權推翻州長下達的居家防疫命令，他說有絕對權力來主導重啟美國經濟。
新聞的種類: 經濟
體育 : 0.0 %
醫藥 : 0.0 %
藝術 : 0.0 %
交通 : 0.0 %
教育 : 0.0 %
計算機 : 0.0 %
環境 : 0.0 %
軍事 : 0.0 %
政治 : 0.24 %
經濟 : 99.76 %


In [14]:
list(zip([1, 2, 3],
    [4, 5, 6]))

[(1, 4), (2, 5), (3, 6)]

In [ ]:
sorted([(3, "a"), ()])